In [1]:
import sqlite3

Создаём БД, в которой хотим хранить базовый список, списки для разных языков, а также информацию о правках

In [2]:
conn = sqlite3.connect('Swadesh.db')
cur = conn.cursor()

In [3]:
cur.execute('''
CREATE TABLE IF NOT EXISTS "base_list" (
"id" INTEGER NOT NULL UNIQUE,
"word" TEXT NOT NULL UNIQUE,
PRIMARY KEY("id")
);
''')

In [4]:
cur.execute('''
CREATE TABLE IF NOT EXISTS "editors" (
"id" INTEGER NOT NULL UNIQUE,
"username" TEXT NOT NULL,
"telegram_id" INTEGER NOT NULL UNIQUE,
PRIMARY KEY("id" AUTOINCREMENT)
);
''')

In [5]:
cur.execute('''
CREATE TABLE IF NOT EXISTS "edits" (
"id" INTEGER NOT NULL UNIQUE,
"user_id" INTEGER NOT NULL,
"state" INTEGER NOT NULL,
"language_group" TEXT NOT NULL,
"line" INTEGER NOT NULL,
PRIMARY KEY("id" AUTOINCREMENT)
);
''')

Теперь наполняем её собственно списками языков, к сожалению, этот процесс придётся проводить для каждой группы отдельно в силу неоднородности википедийных таблиц

In [6]:
import requests
from bs4 import BeautifulSoup
import re

In [7]:
session = requests.session()

Базовый список

In [8]:
url = 'https://en.wiktionary.org/wiki/Appendix:English_Swadesh_list'
response = session.get(url)
page = response.text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find_all('table')[0].find_all('tr')

tbody = []

for body in table:
    tbody.append(body.find_all('td'))

for body in tbody[1:]:
#     print(body[1].text[:-1])
    cur.execute(f"""
    INSERT INTO base_list
    (word)
    VALUES
    ('{body[1].text[:-1]}');
    """)

conn.commit()

Кельтские языки (разбиты на три таблицы, есть реконструкции):

In [9]:
for pages in ['', '/92-146', '/147-207']:
    group = 'Celtic'
    url = f'https://en.wiktionary.org/wiki/Appendix:{group}_Swadesh_lists' + pages
    response = session.get(url)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    table = soup.find_all('table')[1].find_all('tr')
    thead = [head.text.split('edit')[0].replace('\n', '') for head in table[0].find_all('th')[3:]]
    cur.execute(f'''CREATE TABLE IF NOT EXISTS "{group}" ("id" INTEGER NOT NULL UNIQUE, "state"	INTEGER NOT NULL, PRIMARY KEY("id" AUTOINCREMENT));''')
    try:
        for c in thead:
            cur.execute(f'''ALTER TABLE "{group}" ADD COLUMN "{c}" TEXT;''')
    except:
        pass
    
    tbody = []

    for body in table[1:]:
        line = [element.text.replace('\n', ' 0').replace(', ', ' 0  ') for element in body.find_all('td')[4:]]
        tbody.append(line)
    
    for line in tbody:
        cur.execute(f"""
        INSERT INTO {group}
        ({', '.join([f'"{c}"' for c in thead]) + ''', "state"'''})
        VALUES
        ({', '.join([f'"{c}"' for c in line]) + ', -1'});
        """)
        
conn.commit()

Балтийские языки (ничего сложного, опыт обработки пустых ячеек в таблице):

In [10]:
group = 'Baltic'
url = f'https://en.wiktionary.org/wiki/Appendix:{group}_Swadesh_lists'
response = session.get(url)
page = response.text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find_all('table')[0].find_all('tr')

thead = [head.text.split('edit')[0].replace('\n', '') for head in table[0].find_all('th')[1:]]
cur.execute(f'''CREATE TABLE IF NOT EXISTS "{group}" ("id" INTEGER NOT NULL UNIQUE, "state"	INTEGER NOT NULL, PRIMARY KEY("id" AUTOINCREMENT));''')
try:
    for c in thead:
        cur.execute(f'''ALTER TABLE "{group}" ADD COLUMN "{c}" TEXT;''')
except:
    pass

tbody = []

for body in table[1:]:
    line = [re.sub(r'\s\"?\([a-z]+\)\"?', '', element.text
                   .replace('-\n', '')
                   .replace('\n', ' 0')
                   .replace(', ', ' 0  '))
            for element in body.find_all('td')[2:]]
    tbody.append(line)
    
for line in tbody:
    cur.execute(f"""
    INSERT INTO {group}
    ({', '.join([f'"{c}"' for c in thead]) + ''', "state"'''})
    VALUES
    ({', '.join([f'"{c}"' for c in line]) + ', -1'});
    """)

conn.commit()

Славянские языки (черновым вариантом решаю оставить оба вхождения слова для языков с кирилическими алфавитами)

In [11]:
group = 'Slavic'
url = f'https://en.wiktionary.org/wiki/Appendix:{group}_Swadesh_lists'
response = session.get(url)
page = response.text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find_all('table')[0].find_all('tr')

thead = [re.sub('\( | \)', '', head
        .get_text(separator=' ')
        .split('edit')[0])
        .replace('  ', '') 
        for head in table[0].find_all('th')[4:]]
cur.execute(f'''CREATE TABLE IF NOT EXISTS "{group}" ("id" INTEGER NOT NULL UNIQUE, "state"	INTEGER NOT NULL, PRIMARY KEY("id" AUTOINCREMENT));''')
try:
    for c in thead:
        cur.execute(f'''ALTER TABLE "{group}" ADD COLUMN "{c}" TEXT;''')
except:
    pass

tbody = []

for body in table[1:]:
    line = [element.text
            .replace('*', '')
            .replace(' (', ', ')
            .replace(', ', ' 0  ')
            .replace(')', '')
            .replace('-', '')
            + ' 0'
            for element in body.find_all('td')[4:]]
    tbody.append(line)
    
for line in tbody:
    cur.execute(f"""
    INSERT INTO {group}
    ({', '.join([f'"{c}"' for c in thead]) + ''', "state"'''})
    VALUES
    ({', '.join([f'"{c}"' for c in line]) + ', -1'});
    """)

conn.commit()

Прибалтийско-финские языки

In [12]:
group = 'Finnic'
url = f'https://en.wiktionary.org/wiki/Appendix:{group}_Swadesh_lists'
response = session.get(url)
page = response.text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find_all('table')[0].find_all('tr')

thead = [re.sub('\( | \)', '', head
        .get_text(separator=' ')
        .split('edit')[0])
        .replace('  ', '') 
        for head in table[0].find_all('th')[2:]]
cur.execute(f'''CREATE TABLE IF NOT EXISTS "{group}" ("id" INTEGER NOT NULL UNIQUE, "state"	INTEGER NOT NULL, PRIMARY KEY("id" AUTOINCREMENT));''')
try:
    for c in thead:
        cur.execute(f'''ALTER TABLE "{group}" ADD COLUMN "{c}" TEXT;''')
except:
    pass

tbody = []

for body in table[1:]:
    line = [element.text.replace(', ', ' 0  ').replace('*', '').replace('-', '') + ' 0' for element in body.find_all('td')[2:]]
    tbody.append(line)
    
for line in tbody:
    cur.execute(f"""
    INSERT INTO {group}
    ({', '.join([f'"{c}"' for c in thead]) + ''', "state"'''})
    VALUES
    ({', '.join([f'"{c}"' for c in line]) + ', -1'});
    """)

conn.commit()

Тюркские языки

In [13]:
group = 'Turkic'
url = f'https://en.wiktionary.org/wiki/Appendix:{group}_Swadesh_lists'
response = session.get(url)
page = response.text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find_all('table')[0].find_all('tr')

thead = [re.sub('\( | \)', '', head
        .get_text(separator=' ')
        .split('edit')[0])
        .replace('  ', '') 
        for head in table[0].find_all('th')[3:]]
cur.execute(f'''CREATE TABLE IF NOT EXISTS "{group}" ("id" INTEGER NOT NULL UNIQUE, "state"	INTEGER NOT NULL, PRIMARY KEY("id" AUTOINCREMENT));''')
try:
    for c in thead:
        cur.execute(f'''ALTER TABLE "{group}" ADD COLUMN "{c}" TEXT;''')
except:
    pass

tbody = []

for body in table[1:]:
    line = [re.sub(r'\(.+?\)', '', element.text
            .replace(', ', ' 0  ')
            .replace('*', '')
            .replace('-', '')
            + ' 0')
            for element in body.find_all('td')[3:]]
    tbody.append(line)
    
for line in tbody:
    cur.execute(f"""
    INSERT INTO {group}
    ({', '.join([f'"{c}"' for c in thead]) + ''', "state"'''})
    VALUES
    ({', '.join([f'"{c}"' for c in line]) + ', -1'});
    """)

conn.commit()